In [1]:
import requests
import pandas as pd
import numpy as np
import pickle
import time
import random

In [2]:
key = open('key.txt').read()

In [3]:
app_ids = [1397950, 1397960, 1398100]
# 1863540, 1061400 don't have English supported, that is why they fail

In [4]:
categories = [
    ['name'],
    ['type'],
    ['steam_appid'],
    ['developers'],
    ['publishers'],
    ['is_free'],
    ['price_overview', 'initial'],
    ['achievements', 'total'],
    ['release_date', 'date'],
    ['metacritic', 'score']
]

filters = ''
for cat in categories:
    filters += cat[0] + ','
filters += 'basic,genres,categories'

In [5]:
def fetch(game_info, categories):
    info = {}
    for category in categories:
        cur_info = game_info
        failed = False
        
        for subpart in category:
            if not failed and subpart in cur_info.keys():
                cur_info = cur_info[subpart]
            else:
                failed = True
        
        if not failed:
            info[' '.join(category)] = cur_info
        else:
            info[' '.join(category)] = np.NaN
    
    if 'genres' in game_info.keys():
        for genre in game_info['genres']:
            info['Genre: ' + genre['description']] = True
    if 'categories' in game_info.keys():
        for genre in game_info['categories']:
            info['Category: ' + genre['description']] = True
    
    
    return info

In [6]:
def store_query(app_id):
    global filters, key
    
    max_attempts = 11
    attempts = 0
    
    URL = f"http://store.steampowered.com/api/appdetails/?appids={app_id}"
    PARAMS = {'filters': filters, 'l': 'en', 'key': key} # probably don't actually need the key but maybe it will help
    
    while attempts < max_attempts:
        re = requests.get(url=URL, params=PARAMS)

        if re.status_code != 429:
            break
            
        #print(f"waiting on store query for {app_id} ... (attempt {attempts}/{max_attempts})")
        #time.sleep((2 ** attempts) + random.random())
        time.sleep(30 + random.random())
        attempts = attempts + 1
        
    if attempts >= max_attempts:
        print(f"ERROR: Store query timeout on {app_id}. Copy this entire message into failedIDs.txt")
        return None
    else:
        return re
    
def review_query(app_id):
    max_attempts = 11
    attempts = 0
    
    URL = f"http://store.steampowered.com/appreviews/{app_id}?json=1"
    PARAMS = {'num_per_page': 0, 'language': 'all', 'purchase_type': 'all'}
    
    while attempts < max_attempts:
        re = requests.get(url=URL, params=PARAMS)

        if re.status_code != 429:
            break
            
        #print(f"waiting on review query for {app_id} ... (attempt {attempts}/{max_attempts})")
        #time.sleep((2 ** attempts) + random.random())
        time.sleep(30 + random.random())
        attempts = attempts + 1
        
    if attempts >= max_attempts:
        print(f"ERROR: Review query timeout on {app_id}. Copy this entire message into failedIDs.txt")
        return None
    else:
        return re

In [7]:
all_info = []

for app_id in app_ids:

    # make main api query
    re = store_query(app_id)

    try:
        if re is not None and re.json()[str(app_id)]['success']:
            game_info = re.json()[str(app_id)]['data']
            info = fetch(game_info, categories)
            # make review api query
            re = review_query(app_id)
            
            if re is not None:
                summary = re.json()
                if 'query_summary' in summary.keys():
                    q_summary = summary['query_summary']

                    if 'total_positive' in q_summary:
                        info['total_positive'] = q_summary['total_positive']
                    else:
                        info['total_positive'] = np.NaN

                    if 'total_reviews' in q_summary:
                        info['total_reviews'] = q_summary['total_reviews']
                    else:
                        info['total_reviews'] = np.NaN


            all_info.append(info)
    except:
        print(f"ERROR: Epic fail on {app_id}. Copy this entire message into failedIDs.txt")
        
        

df = pd.DataFrame.from_dict(all_info)
df.to_pickle(f"dfFailed.pkl")

1863540, 1061400

In [10]:
df.head()

,name,type,steam_appid,developers,publishers,is_free,price_overview initial,achievements total,release_date date,metacritic score,Genre: Adventure,Genre: Indie,Genre: RPG,Category: Single-player,total_positive,total_reviews,Genre: Casual,Category: Steam Achievements
0,The Scarlet Capital 绯红之都,game,1397950,[飞羽工作室],[飞羽工作室],False,NaN,NaN,"Oct 22, 2020",NaN,True,True,True,True,0,0,NaN,NaN
1,One Night on Halloween,game,1397960,[Cult Shock Games],[Cult Shock Games],False,199.0,15.0,"Oct 29, 2020",NaN,NaN,True,NaN,True,1,1,True,NaN
2,Orwell's Animal Farm,game,1398100,[Nerial],[The Dairymen],False,999.0,5.0,"Dec 10, 2020",NaN,True,True,NaN,True,100,124,NaN,True
